In [1]:
import pandas as pd
import numpy as np

## Format into what ART takes

In [2]:
df = pd.read_csv('clean.csv')
df["Subject"] = df["subject_id"].apply(lambda x: x[4:])

In [3]:
df["density"].replace({"low": 1, "medium": 2, "high": 3}, inplace=True)
df["technique"].replace({"ConeExpand": 1, "2DMiniMap+": 2, "3DMiniMap": 3, "GravityZone+": 4}, inplace=True)

In [4]:
searchdf = df[df["trial_type"] == "Search"].groupby(["Subject", "technique", "density"]).mean().reset_index()[["Subject", "technique", "density", "trial_time"]]
repeatdf = df[df["trial_type"] == "Repeat"].groupby(["Subject", "technique", "density"]).mean().reset_index()[["Subject", "technique", "density", "trial_time"]]

In [5]:
searchdf.Subject.unique()

array(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2',
       '20', '21', '22', '23', '24', '25', '3', '4', '5', '6', '7', '8',
       '9'], dtype=object)

In [6]:
searchdf.to_csv("search.csv", index=False)
repeatdf.to_csv("repeat.csv", index=False)

## Process using ART and re-open

In [7]:
searchdf = pd.read_csv("search.art.csv")
repeatdf = pd.read_csv("repeat.art.csv")

In [8]:
searchdf.columns

Index(['Subject', 'technique', 'density', 'trial_time',
       'aligned(trial_time) for technique', 'aligned(trial_time) for density',
       'aligned(trial_time) for technique*density',
       'ART(trial_time) for technique', 'ART(trial_time) for density',
       'ART(trial_time) for technique*density'],
      dtype='object')

In [9]:
# reogranize into what SPSS takes

search_technique_spss_lines = []
search_density_spss_lines = []
search_interaction_spss_lines = []

for sub_idx, s in enumerate(searchdf.Subject.unique(), start=1):
    subject_row_technique = {"subjedct_id": sub_idx}
    subject_row_density = {"subjedct_id": sub_idx}
    subject_row_interaction = {"subjedct_id": sub_idx}

    for technique in searchdf.technique.unique():
        for density in searchdf.density.unique():

            pdf = searchdf[
                (searchdf["technique"] == technique) &
                (searchdf["density"] == density) &
                (searchdf["Subject"] == s)
            ]

            subject_row_technique["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for technique"].mean()
            subject_row_density["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for density"].mean()
            subject_row_interaction["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for technique*density"].mean()

    search_technique_spss_lines.append(subject_row_technique)
    search_density_spss_lines.append(subject_row_density)
    search_interaction_spss_lines.append(subject_row_interaction)

search_technique_spss = pd.DataFrame(search_technique_spss_lines)
search_density_spss = pd.DataFrame(search_density_spss_lines)
search_interaction_spss = pd.DataFrame(search_interaction_spss_lines)

search_technique_spss.to_csv("search_technique_spss.csv", index=False)
search_density_spss.to_csv("search_density_spss.csv", index=False)
search_interaction_spss.to_csv("search_interaction_spss.csv", index=False)

In [11]:
# reogranize into what SPSS takes

repeat_technique_spss_lines = []
repeat_density_spss_lines = []
repeat_interaction_spss_lines = []

for sub_idx, s in enumerate(repeatdf.Subject.unique(), start=1):
    subject_row_technique = {"subjedct_id": sub_idx}
    subject_row_density = {"subjedct_id": sub_idx}
    subject_row_interaction = {"subjedct_id": sub_idx}

    for technique in repeatdf.technique.unique():
        for density in repeatdf.density.unique():

            pdf = repeatdf[
                (repeatdf["technique"] == technique) &
                (repeatdf["density"] == density) &
                (repeatdf["Subject"] == s)
            ]

            subject_row_technique["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for technique"].mean()
            subject_row_density["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for density"].mean()
            subject_row_interaction["_".join([str(technique), str(density)])] = pdf["ART(trial_time) for technique*density"].mean()

    repeat_technique_spss_lines.append(subject_row_technique)
    repeat_density_spss_lines.append(subject_row_density)
    repeat_interaction_spss_lines.append(subject_row_interaction)

repeat_technique_spss = pd.DataFrame(repeat_technique_spss_lines)
repeat_density_spss = pd.DataFrame(repeat_density_spss_lines)
repeat_interaction_spss = pd.DataFrame(repeat_interaction_spss_lines)

repeat_technique_spss.to_csv("repeat_technique_spss.csv", index=False)
repeat_density_spss.to_csv("repeat_density_spss.csv", index=False)
repeat_interaction_spss.to_csv("repeat_interaction_spss.csv", index=False)